# Query Text Data workflow with "Lazy GraphRAG" retrieval mechanism

Concept:

Vector-based semantic search is a "best first" search approach for relevant chunks - it simply takes the top k most similar chunks based on embedding distances. It has no sense of the breadth of semantic content in the dataset.
Full [GraphRAG](https://aka.ms/graphrag) is a "breadth first" search approach for relevant content in community summaries - it always queries each community. It has no sense of which communities are best.
ming rapid relevance tests on these summaries in rank order to confirm that only the best community summaries from across the breadth of the dataset are used for answer generation.

Lazy GraphRAG requires no summarization-based index (which is expensive, especially if the user only has a few queries). Instead, it relies purely on an embedding-based index for input text chunks (cheap, fast) augmented by the concept graph of noun-phrase cooccurrences extracted from these chunks (free, fast). It works as follows:

- Given a query, text chunks are ranked using vector-based semantic search.
- Chunk rankings are used to determine community rankings in the concept graph.
- In community rank order, the top k chunks are subject to a rapid relevant test (cheap, fast). If relevant chunks are found, the community is retained, otherwise it is discarded.
- After n communities fail to return any relevant chunks, the "iterative narrowing" process repeats with remaining communities in rank order.
- The process terminates when no more testable chunks exist or a predefined testing budget is reached (e.g., 100 relevance tests).

The advantages of this approach include:

- A balance of best-first (semantic search) and breadth-first (community structure) approaches for any testing budget
- Very fast and almost free - suitable for one-off and exploratory queries before committing to a full indexing
- Easily accommodates dynamic contexts where new texts/chunks are streaming in to the dataset (supported in the current implementation)
- Provides a consistent level of "effort" for a fixed test budget, providing various standards of answers to questions (e.g., global questions) for which there is no ground truth (e.g., 1000 chunks = gold standard, 500 chunks = silver standard, 100 chunks = bronze standard, etc.)


In [1]:
import sys

sys.path.append("..")

In [8]:
import os
import asyncio
import io
import pandas as pd
from toolkit.AI.openai_configuration import OpenAIConfiguration
from toolkit.AI.openai_embedder import OpenAIEmbedder
from toolkit.helpers.constants import CACHE_PATH

import toolkit.query_text_data.input_processor as input_processor
import toolkit.query_text_data.relevance_assessor as relevance_assessor
import toolkit.query_text_data.answer_builder as answer_builder
import toolkit.query_text_data.graph_builder as graph_builder
import toolkit.query_text_data.pattern_detector as pattern_detector
import toolkit.graph.graph_fusion_encoder_embedding as gfee
import toolkit.query_text_data.helper_functions as helper_functions
import nest_asyncio # Necessary to run async code in ipynb

nest_asyncio.apply()

In [3]:
# Comment/uncomment the following lines to toggle access to library code updates
import importlib
importlib.reload(input_processor)
importlib.reload(pattern_detector)
importlib.reload(helper_functions)
importlib.reload(relevance_assessor)
importlib.reload(answer_builder)
importlib.reload(graph_builder)

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


<module 'toolkit.query_text_data.graph_builder' from 'c:\\Users\\daedge\\Code\\ITK\\intelligence-toolkit\\example_notebooks\\..\\toolkit\\query_text_data\\graph_builder.py'>

In [4]:
# Set up the AI model and embedding model
ai_configuration = OpenAIConfiguration(
    {
        "api_type": "OpenAI",
        "api_key": os.environ["OPENAI_API_KEY"],
        "model": "gpt-4o-2024-08-06",
    }
)

text_embedder = OpenAIEmbedder(
    configuration=ai_configuration,
)

In [22]:
# Provide text inputs as rows of a dataframe
input_path = "../example_outputs/query_text_data/news_articles/news_articles_texts.csv"
file_name = input_path.split("/")[-1]
file_to_chunks = input_processor.process_file_bytes({file_name: open(input_path, 'rb').read()}, input_processor.PeriodOption.NONE)
print("Chunked input text")

TypeError: unhashable type: 'slice'

In [16]:
max_cluster_size = 25 # The maximum number of concepts in a leaf-level community in the concept graph
# Process the chunks into the index data structures
(
    cid_to_text,
    text_to_cid,
    period_concept_graphs,
    community_to_concepts,
    concept_to_community,
    concept_to_cids,
    cid_to_concepts,
    previous_cid,
    next_cid,
    period_to_cids,
    node_period_counts,
    edge_period_counts,
    hierarchical_communities
) = input_processor.process_chunks(file_to_chunks=file_to_chunks, max_cluster_size=max_cluster_size)
print(f"Processed chunks")

Processed chunks


In [17]:
# Embed the text chunks
cid_to_vector = await helper_functions.embed_texts(
    cid_to_text=cid_to_text,
    text_embedder=text_embedder,
)
print(f"Embedded chunk text")

100%|██████████| 501/501 [00:07<00:00, 68.29it/s] 


Got 0 existing texts
Got 501 new texts
Embedded chunk text


In [18]:
question = "What events are discussed?"
relevance_test_budget = 50 # How many relevance tests are permitted per query. Higher values may provide higher quality results at higher cost
adjacent_search_steps = 1 # How many chunks before and after each relevant chunk to test, once the relevance test budget is near or the search process has terminted
community_relevance_tests = 5 # How many relevance tests to run on each community in turn
relevance_test_batch_size = 5 # How many relevance tests to run in parallel at a time
irrelevant_community_restart = 3 # When to restart testing communities in relevance order

# Mine relevant chunks to the question
async def mine_relevant_chunks():
    (
        relevant_cids,
        chunk_progress,
    ) = await relevance_assessor.detect_relevant_chunks(
        ai_configuration=ai_configuration,
        question=question,
        cid_to_text=cid_to_text,
        cid_to_concepts=cid_to_concepts,
        cid_to_vector=cid_to_vector,
        community_to_concepts=community_to_concepts,
        concept_to_community=concept_to_community,
        previous_cid=previous_cid,
        next_cid=next_cid,
        embedder=text_embedder,
        embedding_cache=CACHE_PATH,
        select_logit_bias=5,
        adjacent_search_steps=adjacent_search_steps,
        relevance_test_budget=relevance_test_budget,
        community_relevance_tests=community_relevance_tests,
        relevance_test_batch_size=relevance_test_batch_size,
        irrelevant_community_restart=irrelevant_community_restart,
    )
    return relevant_cids, chunk_progress

if __name__ == "__main__":
    relevant_cids, chunk_progress = await mine_relevant_chunks()
    print(f"Mined relevant chunks")
    


New loop after 0 tests
Assessing relevance for topic 92 with 5 chunks


100%|██████████| 5/5 [00:00<00:00,  5.74it/s]


Batch 1 of 1: 5 relevant chunks
Community 92: True
Assessing relevance for topic 68 with 5 chunks


100%|██████████| 5/5 [00:01<00:00,  4.57it/s]


Batch 1 of 1: 5 relevant chunks
Community 68: True
Assessing relevance for topic 93 with 5 chunks


...

Batch 1 of 1: 0 relevant chunks
Community 93: False
Assessing relevance for topic 94 with 5 chunks


0it [00:00, ?it/s]


Batch 1 of 1: 0 relevant chunks
Community 94: False
Assessing relevance for topic 158 with 5 chunks


0it [00:00, ?it/s]

Batch 1 of 1: 0 relevant chunks
Community 158: False
0 successive irrelevant communities; breaking
Assessing relevance for neighbours with 0 chunks
[('topic 68', 96, 'Yes'), ('topic 68', 324, 'Yes'), ('topic 68', 144, 'Yes')]
**Chunks relevant/tested: 10/10** (topic 92: 5/5; topic 68: 5/5)
Mined relevant chunks


In [19]:
answer_batch_size = 10 # How many chunks to integrate into the answer at a time
# Generate an extended answer to the question, which could then be summarized
(
    partial_answers,
    answer_progress,
) = answer_builder.answer_question(
    ai_configuration=ai_configuration,
    question=question,
    relevant_cids=relevant_cids,
    cid_to_text=cid_to_text,
    answer_batch_size=answer_batch_size,
)
print(f"Answered question")

[{'role': 'system', 'content': 'You are a helpful assistant tasked with creating a JSON object that answers a given user question.\n\nThe output object should update the input JSON object in a way that:\n\n- answers the user question as directly and comprehensively as possible using relevant information from the text chunks\n- represents a single coherent narrative, rather than a collection of unrelated facts. Input texts that are clearly unrelated should be ignored\n- incudes a title and structured headings as appropriate\n- includes prose between all headings to explain the context and significance of the information presented\n- includes bridging text that makes connections between topic areas, explaining how they relate to each other\n- supports each sentence with a source reference to the file and text chunk: "[source: <file> (<chunk_id>), <file> (<chunk_id>)]. Always use the full name of the file - do not abbreviate - and enter the full filename before each chunk id, even if the 

In [20]:
# Output some process summarry information
print(chunk_progress)
print(answer_progress)

**Chunks relevant/tested: 10/10** (topic 92: 5/5; topic 68: 5/5)
**Chunks referenced/relevant: 8/10** (answer 1: 8/10)


In [21]:
# Output the final extended answer
final_answer = partial_answers[-1]
print(final_answer)

# Key Global Events: Economic and Climate Forums

*In response to: What events are discussed?*

## Introduction

This report provides an overview of significant global events focused on economic and climate issues. The events are grouped into two main categories: Economic Forums and Climate Summits.

**Economic Forums:**
- Global Economic Forum in the Financial District
- World Economic Forum in New York City
- Global Economic Forum in New York
- World Economic Forum at the International Conference Center

**Climate Summits:**
- Climate Summit in Beijing
- Global Climate Action Forum in Berlin
- Global Climate Forum in Sydney
- United Nations Climate Summit

## Analysis

### Global Economic Forum in the Financial District

The Global Economic Forum convened in the Financial District on March 21, 2024, bringing together economists and global leaders to discuss future economic trends and challenges. Key topics included the impact of technological advancements, sustainable economic polici